<h1>Coursera Capstone Data Collision</h1>

This notebook is the final capstone of the IBM Data Science Coursera course. With this notebook, the <b>Data Collisions - All years</b> data set for seattle will be evaluated and a prediction will be done for the probability of a collission under some given circumstances.

To achieve this, the data will be analyzed, avaluated, preprocessed and models will be created and tested to find the best model and prediction. 

<h2>Business Problem</h2>

There are several variables who can indicate that there is a high risk to have an accident, variables which influence the risk could be for example the weather condition (are the roads slippy?), time of the day (is the driver tired?) and so on. 
With the following examination we want to find out what the key indicators are to predict an accident and the severity of it. A modell will be created with the best fitting method that could be used by navigation systems to warn the driver if the
probability of an accident is high.

<h2>Data</h2>

We will use the data set recommended for this course (<b>Data Collisions - All years</b> for Seattle 2004 to now). As a first step we create a dataframe with the raw data, analyzing the given properties.

In [19]:
import pandas as pd
import numpy as np
import wget
wget.download('https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv')
df = pd.read_csv('Data-Collisions.csv')
df.head()

100% [........................................................................] 73917638 / 73917638

C:\Users\js423\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [21]:
df.describe()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,INTKEY,SEVERITYCODE.1,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,SDOT_COLCODE,SDOTCOLNUM,SEGLANEKEY,CROSSWALKKEY
count,194673.000000,189339.000000,189339.000000,194673.000000,194673.000000,194673.000000,65070.000000,194673.000000,194673.000000,194673.000000,194673.000000,194673.000000,194673.000000,1.149360e+05,194673.000000,1.946730e+05
mean,1.298901,-122.330518,47.619543,108479.364930,141091.456350,141298.811381,37558.450576,1.298901,2.444427,0.037139,0.028391,1.920780,13.867768,7.972521e+06,269.401114,9.782452e+03
std,0.457778,0.029976,0.056157,62649.722558,86634.402737,86986.542110,51745.990273,0.457778,1.345929,0.198150,0.167413,0.631047,6.868755,2.553533e+06,3315.776055,7.226926e+04
min,1.000000,-122.419091,47.495573,1.000000,1001.000000,1001.000000,23807.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.007024e+06,0.000000,0.000000e+00
25%,1.000000,-122.348673,47.575956,54267.000000,70383.000000,70383.000000,28667.000000,1.000000,2.000000,0.000000,0.000000,2.000000,11.000000,6.040015e+06,0.000000,0.000000e+00
50%,1.000000,-122.330224,47.615369,106912.000000,123363.000000,123363.000000,29973.000000,1.000000,2.000000,0.000000,0.000000,2.000000,13.000000,8.023022e+06,0.000000,0.000000e+00
75%,2.000000,-122.311937,47.663664,162272.000000,203319.000000,203459.000000,33973.000000,2.000000,3.000000,0.000000,0.000000,2.000000,14.000000,1.015501e+07,0.000000,0.000000e+00
max,2.000000,-122.238949,47.734142,219547.000000,331454.000000,332954.000000,757580.000000,2.000000,81.000000,6.000000,2.000000,12.000000,69.000000,1.307202e+07,525241.000000,5.239700e+06


A first raw analyzes shows that we have 194.673 rows with unique accidents in the dataset.

The dataset already contains a row named SEVERITYCODE which will be our target dependend variable to predict the severity of an incident.

Regarding the metadata of the dataset the SEVERITYCODE property is categorical property with the following values:
<ul>
    <li>3—fatality</li>
    <li>2b-serious injury</li>
    <li>2-injury</li>
    <li>1-prop damage</li>
    <li>0-unknown</li>
</ul>

By knowing this we'd prefer a classification machine learning algorithm that supports discrete values.

As a first sanity check we'll list the occurence of diffferent SEVERITYCODE values in the dataset.


In [30]:
print("SEVERITY")
df['SEVERITYCODE'].value_counts(dropna=False)

SEVERITY


1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

We see that only the values for <b>prop damage</b> and <b>injury</b> are used, and that accidents with a lower severity occurs more often.

<h2>Data Preparation</h2>

As a first step we will remove columns that we will not use in our examinations. These are for example columns where a description is missing in the metadata and where for this reason it is unclear what they are about, also, columns were we think that they will not be useful to predict the severity will be removed.

In [32]:
print('Existing columns:')
list(df.columns)

Existing columns:


['SEVERITYCODE',
 'X',
 'Y',
 'OBJECTID',
 'INCKEY',
 'COLDETKEY',
 'REPORTNO',
 'STATUS',
 'ADDRTYPE',
 'INTKEY',
 'LOCATION',
 'EXCEPTRSNCODE',
 'EXCEPTRSNDESC',
 'SEVERITYCODE.1',
 'SEVERITYDESC',
 'COLLISIONTYPE',
 'PERSONCOUNT',
 'PEDCOUNT',
 'PEDCYLCOUNT',
 'VEHCOUNT',
 'INCDATE',
 'INCDTTM',
 'JUNCTIONTYPE',
 'SDOT_COLCODE',
 'SDOT_COLDESC',
 'INATTENTIONIND',
 'UNDERINFL',
 'WEATHER',
 'ROADCOND',
 'LIGHTCOND',
 'PEDROWNOTGRNT',
 'SDOTCOLNUM',
 'SPEEDING',
 'ST_COLCODE',
 'ST_COLDESC',
 'SEGLANEKEY',
 'CROSSWALKKEY',
 'HITPARKEDCAR']

In [36]:
columns_to_remove = [
    'OBJECTID', 'REPORTNO', 'STATUS', "OBJECTID", "INCKEY", "COLDETKEY", "INTKEY", "LOCATION", "SEVERITYCODE.1", "SEVERITYDESC", 
    "SDOT_COLDESC", "SDOTCOLNUM", "ST_COLDESC", "SEGLANEKEY", "CROSSWALKKEY", "X", "Y"
]
df.drop(columns_to_remove, axis=1)

,SEVERITYCODE,ADDRTYPE,EXCEPTRSNCODE,EXCEPTRSNDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,...,SDOT_COLCODE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLCODE,HITPARKEDCAR
0,2,Intersection,,NaN,Angles,2,0,0,2,2013/03/27 00:00:00+00,...,11,NaN,N,Overcast,Wet,Daylight,NaN,NaN,10,N
1,1,Block,NaN,NaN,Sideswipe,2,0,0,2,2006/12/20 00:00:00+00,...,16,NaN,0,Raining,Wet,Dark - Street Lights On,NaN,NaN,11,N
2,1,Block,NaN,NaN,Parked Car,4,0,0,3,2004/11/18 00:00:00+00,...,14,NaN,0,Overcast,Dry,Daylight,NaN,NaN,32,N
3,1,Block,,NaN,Other,3,0,0,3,2013/03/29 00:00:00+00,...,11,NaN,N,Clear,Dry,Daylight,NaN,NaN,23,N
4,2,Intersection,NaN,NaN,Angles,2,0,0,2,2004/01/28 00:00:00+00,...,11,NaN,0,Raining,Wet,Daylight,NaN,NaN,10,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194668,2,Block,,NaN,Head On,3,0,0,2,2018/11/12 00:00:00+00,...,11,NaN,N,Clear,Dry,Daylight,NaN,NaN,24,N
194669,1,Block,,NaN,Rear Ended,2,0,0,2,2018/12/18 00:00:00+00,...,14,Y,N,Raining,Wet,Daylight,NaN,NaN,13,N
194670,2,Intersection,,NaN,Left Turn,3,0,0,2,2019/01/19 00:00:00+00,...,11,NaN,N,Clear,Dry,Daylight,NaN,NaN,28,N
194671,2,Intersection,,NaN,Cycles,2,0,1,1,2019/01/15 00:00:00+00,...,51,NaN,N,Clear,Dry,Dusk,NaN,NaN,5,N
